In [10]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━

In [2]:
# 로드
import time
import datetime
import os
import copy
import cv2
import random
import numpy as np
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.optim import lr_scheduler
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from PIL import Image
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm
import torch.optim as optim

In [3]:
#transforms.Compose : Rescale 과 RandomCrop 을 한번에 수행
#Rescale: 이미지의 크기를 조절
#RandomCrop: 이미지를 무작위로 자른다
#정규화
def func(x):  #아래 transforms_train = 코드에서 transforms.Lambda(lambda x: x.rotate(90)) 에서 나는 에러를 잡기 위해 def로 빼주고 람다 속 람다를 제거함
    return x.rotate(90)
transforms_train = transforms.Compose([
    transforms.Resize([int(600), int(600)], interpolation=transforms.InterpolationMode.BOX), # interpolation=4 워닝을 제거하기 위해 변형
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.Lambda(func),
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
transforms_val = transforms.Compose([
    transforms.Resize([int(600), int(600)], interpolation=transforms.InterpolationMode.BOX),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
#data_train_path 경로의 이미지를 transforms.Compose 로 정규화한 데이터 기준으로 트랜스폼
# train_data_set = datasets.ImageFolder(data_train_path, transform=transforms_train)
# val_data_set = datasets.ImageFolder(data_validation_path, transform=transforms_val)
# # 변수 선언
# dataloaders, batch_num = {}, {}
# # dataloaders 빈딕셔너리에 train/val 키랑 DataLoder 밸류 넣기
# # DataLoader로 학습용 데이터 준비 : 데이터셋의 특징(feature)을 가져오고 하나의 샘플에 정답(label)을 지정하는 일을 한다
# dataloaders['train'] = DataLoader(train_data_set,
#                                   batch_size=hyper_param_batch,
#                                   shuffle=True,
#                                   num_workers=0)  # aihub코드  num_workers = 4
# dataloaders['val'] = DataLoader(val_data_set,
#                                 batch_size=hyper_param_batch,
#                                 shuffle=False,
#                                 num_workers=0)  # aihub코드  num_workers = 4
#즉 dataloaders 딕셔너리에는 train / val 이 key 각 밸류는 정규화한 이미지 데이터에 + 라벨이 붙음
#DataLoader를 통해 네트워크에 올리기
#from torch.utils.data import Dataset,DataLoader
#testloader = DataLoader(testset, batch_size=2, shuffle=False, num_workers=0)
    #데이터 로더는 데이터의 대량 가져오기 또는 내보내기를 위한 클라이언트 응용 프로그램
    #for data, target in testloader: 에서 data는 데이터의 특징  target은 데이터의 정답값

In [ ]:
# PATH = '/content/drive/MyDrive/project/scalp_weights/'+'aram_model5.pt'   # 모델경로
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")     # 쿠다 쓸 수 있으면 사용
# model = torch.load(PATH, map_location=device)     # 모델로드
# with torch.no_grad(): # 평가할 땐  gradient를 backpropagation 하지 않기 때문에 no grad로 gradient 계산을 막아서 연산 속도를 높인다
#             for data, target in tqdm(testloader):
#                 data, target  = data.to(device), target.to(device)
#                 output = model(data)   # model1에 데이터를 넣어서 아웃풋 > [a,b,c,d] 각 0,1,2,3 의 확률값 리턴 가장 큰 것이 pred
#                 # output_list.append(output);
#                 test_loss += F.nll_loss(output, target, reduction = 'sum').item()  # test_loss변수에 각 로스를 축적
#                 pred = output.argmax(dim=1, keepdim=True) # argmax : 리스트에서 최댓값의 인덱스를 뽑아줌 > y값아웃풋인덱
#                 correct += pred.eq(target.view_as(pred)).sum().item() # accuracy 측정을 위한 변수 # 각 예측이 맞았는지 틀렸는지 correct변수에 축적 맞을 때마다 +1  # # view_as() 함수는 target 텐서를 view_as() 함수 안에 들어가는 인수(pred)의 모양대로 다시 정렬한다. #  view_as() 함수는 target 텐서를 view_as() 함수 안에 들어가는 인수(pred)의 모양대로 다시 정렬한다. #  pred.eq(data) : pred와 data가 일치하는지 검사
# test_loss /= len(testloader.dataset)  # 로스축적된 로스를 데이터 수(경로안jpg수)로 나누기
# # 로스, 아큐러시 출력 ( :.4f 소수점반올림 )
# print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(test_loss, correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))  # 축적된 예측값을 데이터 개수로 나누기 *100  확률%값

In [4]:
model_name = 'efficientnet-b6'
num_classes = 4  # 여기선 고정값 아웃풋 0 1 2 3 // 4가지

In [21]:
#웹페이지에서 이미지를 업로드 받은 후 연산시간을 줄이기 위해 미리 로드
#PATH
#checkpoint_model2.pt
PATH2 = '/content/drive/MyDrive/capstone/scalp_weights/'+'checkpoint_model2.pt'  # 모델2 피지과다
PATH3 = '/content/drive/MyDrive/capstone/scalp_weights/'+'checkpoint_model3.pt'  # 모델3 모낭사이홍반
PATH6 = '/content/drive/MyDrive/capstone/scalp_weights/'+'checkpoint_model6.pt'  # 모델6 탈모

#cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#모델 로드

model2 = EfficientNet.from_pretrained(model_name, num_classes=num_classes).to(device)  # 모델 다운로드
model3 = EfficientNet.from_pretrained(model_name, num_classes=num_classes).to(device)
model6 = EfficientNet.from_pretrained(model_name, num_classes=num_classes).to(device)

# 저장된 state_dict 로드

checkpoint2 = torch.load(PATH2)
checkpoint3 = torch.load(PATH3)
checkpoint6 = torch.load(PATH6)

# state_dict를 각각의 모델에 로드
model2.load_state_dict(checkpoint2['model_state_dict'])
model3.load_state_dict(checkpoint3['model_state_dict'])
model6.load_state_dict(checkpoint6['model_state_dict'])

#모델을 평가모드로 전환   # 평가모드와 학습모드의 layer 구성이 다르다
model2.eval()
model3.eval()
model6.eval()

Loaded pretrained weights for efficientnet-b6
Loaded pretrained weights for efficientnet-b6
Loaded pretrained weights for efficientnet-b6


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 56, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(56, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        56, 56, kernel_size=(3, 3), stride=[1, 1], groups=56, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(56, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        56, 14, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        14, 56, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        56, 32, kernel_size=(1, 1), stride=(1, 1), bias=False
  

In [13]:
#전처리 : 트랜스폼 규칙 선언 # validation set 의 트랜스폼 규칙과 동일
transforms_test = transforms.Compose([  transforms.Resize([int(600), int(600)], interpolation=transforms.InterpolationMode.BOX),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])    ])
#root 경로 폴더 속 jpg를 전처리, 텐서화 # /content/drive/MyDrive/capstone/
testset = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/capstone/test_data',
                    transform = transforms_test)
#앞서 전처리한 testset 을 DataLoader를 통해 네트워크에 올리기
testloader = DataLoader(testset, batch_size=2, shuffle=False, num_workers=0)
    #데이터 로더는 데이터의 대량 가져오기 또는 내보내기를 위한 클라이언트 응용 프로그램

In [22]:
#for data, target in testloader: 에서 data는 데이터의 특징  target은 데이터의 정답값
with torch.no_grad(): # 평가할 땐  gradient를 backpropagation 하지 않기 때문에 no grad로 gradient 계산을 막아서 연산 속도를 높인다
  for data, target in testloader:
    data, target  = data.to(device), target.to(device)
    output2 = model2(data) # model에 데이터를 넣어서 아웃풋 > [a,b,c,d] 각 0,1,2,3 의 확률값 리턴 가장 큰 것이 pred
    output3 = model3(data)
    output6 = model6(data)
    #predict # # 0~3값만 뽑기
    m2p = output2.argmax(dim=1, keepdim=True)[0][0].tolist()
    m3p = output3.argmax(dim=1, keepdim=True)[0][0].tolist()
    m6p = output6.argmax(dim=1, keepdim=True)[0][0].tolist()

    print(f"Model2 prediction: {m2p}, Model3 prediction: {m3p}, Model6 prediction: {m6p}")

Model2 prediction: 0, Model3 prediction: 0, Model6 prediction: 2


In [24]:
#진단
d_list = [] # 두피유형진단결과
    # # 두피 유형 진단법
    # if m1p == 0 and m2p == 0 and m3p == 0 and m4p == 0 and m5p == 0 and m6p == 0 :
    #     d1 = '정상입니다.'
    #     d_list.append(d1)
    # elif m1p != 0 and m2p == 0 and m3p == 0 and m4p == 0 and m5p == 0 and m6p == 0 :
    #     d2 = '건성 두피입니다.'
    #     d_list.append(d2)
    # elif m1p == 0 and m2p != 0 and m3p == 0 and m4p == 0 and m5p == 0 and m6p == 0 :
    #     d3 = '지성 두피입니다.'
    #     d_list.append(d3)
    # elif m2p == 0 and m3p != 0 and m4p == 0 and m5p == 0 and m6p == 0 :
    #     d4 = '민감성 두피입니다.'
    #     d_list.append(d4)
    # elif m2p != 0 and m3p != 0 and m4p == 0 and m6p == 0 :
    #     d5 = '지루성 두피입니다.'
    #     d_list.append(d5)
    # elif m3p == 0 and m4p != 0 and m6p == 0 :
    #     d6 = '염증성 두피입니다.'
    #     d_list.append(d6)
    # elif m3p == 0 and m4p == 0 and m5p != 0 and m6p == 0 :
    #     d7 = '비듬성 두피입니다.'
    #     d_list.append(d7)
    # elif m1p == 0 and m2p != 0 and m3p == 0 and m4p == 0 and m5p == 0 and m6p != 0 :
    #     d8 = '탈모입니다.'
    #     d_list.append(d8)
    # else:
    #     d9 = '복합성 두피입니다.'
    #     d_list.append(d9)

# 두피 유형 진단법
if m2p == 0 and m3p == 0 and m6p == 0:
    d1 = '정상입니다.'
    d_list.append(d1)
elif m2p != 0 and m3p == 0 and m6p == 0:
    d2 = '지성 두피입니다.'
    d_list.append(d2)
elif m2p == 0 and m3p != 0 and m6p == 0:
    d3 = '민감성 두피입니다.'
    d_list.append(d3)
elif m2p != 0 and m3p != 0 and m6p == 0:
    d4 = '지루성 두피입니다.'
    d_list.append(d4)
elif m2p != 0 and m3p == 0 and m6p != 0:
    d5 = '탈모입니다.'
    d_list.append(d5)
else:
    d6 = '복합성 두피입니다.'
    d_list.append(d6)

# 진단 결과 출력
print(d_list)

['복합성 두피입니다.']
